### Logistic Regression Training on Labeled Data

In [1]:
from gensim.models import Word2Vec
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the pre-trained Word2Vec model
word2vec_model = Word2Vec.load("../../models/kurdish-word2vec-75279.model")

In [2]:
def get_average_word_vector(sentence, model, num_features, stopwords_path="./dataSet/kurdishStopWordsCleaned.txt"):
    """Calculate the average word vector for a sentence, excluding stop words."""
    # Load stop words from file
    with open(stopwords_path, 'r') as file:
        stopwords = set(file.read().split())
    
    words = sentence.split()
    feature_vector = np.zeros((num_features,), dtype="float32")
    num_words = 0
    for word in words:
        if word not in stopwords and word in model.wv.key_to_index:  # Check if word is not a stop word
            print(word)
            num_words += 1
            feature_vector = np.add(feature_vector, model.wv[word])
    return feature_vector / num_words if num_words > 0 else feature_vector

In [3]:
# Load and prepare the dataset
dataset_path = './dataSet/preprocessed/preprocess-40000.csv'
df = pd.read_csv(dataset_path)
X = df['kurdish_tweets'].astype(str)
y = df['sentiment_id']

# Vectorize the text data
X_vectors = np.array([get_average_word_vector(text, word2vec_model, 100) for text in X])

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_vectors, y, test_size=0.2, random_state=42)

# Initialize and train the classifier
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# Evaluate the classifier's performance
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

دەزانم
گوێم
خووی
دەستم
لایەن
ئەوەوە
جێگا
سەرئێشە
ئاهەنگی
زووانە
هاوڕێکانی
دەمانەوێت
بازرگانی
بکەین
بلیتی
ئاهەنگی
حەزی
هاوڕێکانم
دەبێ
ئیم
هاوڕێیەکی
کۆن
هاوسەرگیری
منی
دەوێت
بیرت
دەکەم
ببورن
لانیکەم
ڕۆژی
ناتوانم
بخەوم
خنکا
گۆرانییە
گەمژانە
بدەم
بگەمە
بێئەدەبی
سەیری
گردەکان
بکەیت
لەندەن
ئەوا
تێدەگەیت
گەشتێکە
چونکە
هەفتە
هەفتە
دواکەوتبوو
سەیری
دەکەم
دەستکەوت
لێرەیە
کارەبا
نەماوە
ڕازی
ئەوەندە
خەواڵووی
دیسانەوە
ئەوەندە
درەنگ
جارێکی
شکست
باسی
سەرسام
نەبووە
دزەپێکردنی
بزانن
قەناعەتت
تۆم
پێم
وایە
تازە
هاوڕێیەکی
ترم
لەدەستداوە
ئای
هیوادارم
منیش
کێشەی
خەوتنم
پێم
سەیرە
بەیانی
نهێنییە
خراپەکانم
دەگێڕم
نهێنی
قسەکانی
نەخشەی
بابەتێک
پرۆگرامی
کۆنفرانسی
شێوەی
ئۆنلاین
شتێکم
خوارد
نازانم
بەردەوام
دەگێڕمەوە
خواردن
ئەوەندە
ماندووم
پێموایە
دڵنیاییەوە
تووشی
هەوکردنی
گوێچکە
چوونە
سەر
جێگا
ته
نها
ڕێگام
هەبوونی
گرێبەست
کچانی
تەمەن
خواردنەوەی
جین
پاس
باسی
هەست
پیری
ببورە
خەڵک
ڕەوشت
بهێنن
بزانن
لەوەی
ئەوەندە
دام
لێبدەم
ئەوەندە
ماندوو
ئەف
ڕقم
کاتی
ڕقم
ڕقم
لێیە
ڕقم
لێیە
ڕقم
ئەوەندە
بێزارکەر
نیوەی
شەودا
تایپکردن
ناتو

In [4]:
from joblib import dump
from joblib import load

# Save the trained model to a file
model_filename = './models/LogisticModel40000.joblib'
dump(clf, model_filename)
print(f"Model saved to {model_filename}")

Model saved to ./models/LogisticModel40000.joblib


In [5]:

lr_filename = './models/LogisticModel40000.joblib'
lr_loaded = load(lr_filename)

In [6]:
sentiments = {
    1: {'text': 'بەتاڵ', 'english': 'empty'},
    2: {'text': 'دڵتەنگ', 'english': 'sadness'},
    3: {'text': 'پەرۆش', 'english': 'enthusiasm'},
    4: {'text': 'ئاسایی', 'english': 'neutral'},
    5: {'text': 'نیگەران', 'english': 'worry'},
    6: {'text': 'سوپرایز', 'english': 'surprise'},
    7: {'text': 'خۆشەویستی', 'english': 'love'},
    8: {'text': 'خۆشی', 'english': 'fun'},
    9: {'text': 'ڕق', 'english': 'hate'},
    10: {'text': 'بەختەوەر', 'english': 'happiness'},
    11: {'text': 'بێزار', 'english': 'boredom'},
    12: {'text': 'ئارام', 'english': 'relief'},
    13: {'text': 'تووڕە', 'english': 'anger'}
}

In [11]:
new_text = "دڵتەنگ"
new_vector=get_average_word_vector(new_text, word2vec_model, 100)
lr_prediction = lr_loaded.predict([new_vector])
print("Predicted class:",lr_prediction, sentiments[lr_prediction[0]]['text'])


دڵتەنگ
Predicted class: [2] دڵتەنگ


### Support Vector Machine Training on Labeled Data

In [12]:
from sklearn.svm import SVC
clf=SVC(kernel='linear')
clf.fit(X_train, y_train)

SVC(kernel='linear')

##### Evaluate the classifier's performance


In [13]:
y_pred=clf.predict(X_test)
accuracy=accuracy_score(y_test,y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.28


In [14]:
#saving mode
model_filename = './models/SVM40000.joblib'
dump(clf, model_filename)
print(f"Model saved to {model_filename}")

Model saved to ./models/SVM40000.joblib


In [16]:
#testing svm model

lr_filename = './models/SVM40000.joblib'
lr_loaded = load(lr_filename)
new_text = "ئەوەندە خەواڵووی دیسانەوە و تەنانەت ئەوەندە درەنگ نەبووە. جارێکی تر شکست دێنم."
new_vector=get_average_word_vector(new_text, word2vec_model, 100)
lr_prediction = lr_loaded.predict([new_vector])
print("Predicted class:",lr_prediction, sentiments[lr_prediction[0]]['text'])

ئەوەندە
خەواڵووی
دیسانەوە
ئەوەندە
درەنگ
جارێکی
شکست
Predicted class: [5] نیگەران


### Random Forest Training on Labeled Data

In [5]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the Random Forest classifier
clf = RandomForestClassifier(n_estimators=1000, random_state=42)

# Train the classifier on the training data
clf.fit(X_train, y_train)

# Evaluate the classifier's performance on the test data
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.30


In [7]:
from joblib import dump
from joblib import load
model_filename = './models/RandomForest9000.joblib'
dump(clf, model_filename)
print(f"Model saved to {model_filename}")

Model saved to ./models/RandomForest9000.joblib


In [11]:
lr_filename = './models/RandomForest9000.joblib'
lr_loaded = load(lr_filename)

In [23]:
#testing Random Forest model
new_text = "بێزارم"
new_vector=get_average_word_vector(new_text, word2vec_model, 100)
lr_prediction = lr_loaded.predict([new_vector])
print("Predicted class:",lr_prediction, sentiments[lr_prediction[0]]['text'])

بێزارم
Predicted class: [2] دڵتەنگ
